# 概要
* livedoorニュースの記事に対して Comprehend の topic model を使ってみる

In [ ]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar -zxvf ldcc-20140209.tar.gz
!rm text/CHANGES.txt
!rm text/README.txt
!rm text/topic-news/LICENSE.tx
!rm text/dokujo-tsushin/LICENSE.txt

In [ ]:
import boto3,time,sagemaker
import pandas as pd

In [ ]:
client = boto3.client('comprehend')

In [ ]:
# 設定値
BUCKET = sagemaker.Session().default_bucket()
INPUT_S3_LOCATION = 's3://'+BUCKET+'/comprehend/topic_model/input'
OUTPUT_S3_LOCATION = 's3://'+BUCKET+'/comprehend/topic_model/output'
JOBNAME = 'MySecondJob'
ROLE = 'arn:aws:iam::155580384669:role/comprehend_role'

In [ ]:
%%time
sagemaker.s3.S3Uploader.upload('./text/',INPUT_S3_LOCATION)

In [ ]:
response = client.start_topics_detection_job(
    NumberOfTopics=10,
    InputDataConfig={
        'S3Uri': INPUT_S3_LOCATION,
        'InputFormat': 'ONE_DOC_PER_FILE' # 1ファイルで1ドキュメント扱い、他にはONE_DOC_PER_LINEで1行1ドキュメント扱いも可能
    },
    OutputDataConfig={
        'S3Uri':OUTPUT_S3_LOCATION
    },
    DataAccessRoleArn=ROLE,
    JobName=JOBNAME
)
JOBID = response['JobId']

In [ ]:
OUTPUT_S3_FILE_LOCATION = client.describe_topics_detection_job(JobId=JOBID)['TopicsDetectionJobProperties']['OutputDataConfig']['S3Uri']
print(OUTPUT_S3_FILE_LOCATION)

In [ ]:
sagemaker.s3.S3Downloader.download(OUTPUT_S3_FILE_LOCATION,'./')

In [ ]:
!tar zxvf output.tar.gz

In [ ]:
pd.read_csv('./doc-topics.csv').head()

In [ ]:
pd.read_csv('./topic-terms.csv').head()